# 02. Sentiment Analytics
This tutorial shows you how to use LLMs to perform sentiment analytics with Semantic Kernel.

## Prerequisites

Refer to [01-get-started.ipynb](./01-get-started.ipynb) to install Ollama.

## Install Semantic Kernel

Run the following F# (fsharp) cell to install the packages.

In [8]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"

Installed Packages Microsoft.SemanticKernel, 1.11.1

## Instantiate the kernel

In [9]:
#pragma warning disable SKEXP0010
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

// We use Mistral for this tutorial 
var modelId = "mistral";
// local Ollama endpoint
var endpoint = new Uri("http://localhost:11434");

var kernelBuilder = Kernel.CreateBuilder();
var kernel = kernelBuilder
    .AddOpenAIChatCompletion(
        modelId,
        new Uri("http://localhost:11434"),
        apiKey:null) 
    .Build();

#pragma warning restore SKEXP0010

##  Example code

The following example creates a prompt template and then use it to performance sentiment analytics.

In [12]:
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.TemplateEngine;

var chatService = kernel.GetRequiredService<IChatCompletionService>();

var executionSettings = new OpenAIPromptExecutionSettings
{
    MaxTokens = 2000,
    Temperature = 0,
};

var skPrompt = """

What is the sentiment of the following feedback, which is delimited with triple backticks?

Provide your answer as a single word from the following list ONLY:

- positive
- negative
- netural

Feedback text: ```{{$input}}```
""";

// Create a prompt template configuration
var promptTemplateConfig = new PromptTemplateConfig(skPrompt);

var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);

var renderedPrompt = await promptTemplate.RenderAsync(kernel, new() { ["input"] = "Hello from SemanticKernel" });

Console.WriteLine(renderedPrompt);


What is the sentiment of the following feedback, which is delimited with triple backticks?

Provide your answer as a single word from the following list ONLY:

- positive
- negative
- netural

Feedback text: ```Hello from SemanticKernel```


In [14]:
// Now let's create a function from the prompt template
var sentimentAnalyticsFunc = kernel.CreateFunctionFromPrompt(skPrompt, executionSettings);

var getSentimentTextAsync = async (string text) =>
{
    var response = await kernel.InvokeAsync(sentimentAnalyticsFunc, new() { ["input"] = text });
    return response.GetValue<string>()?.TrimEnd('.');
};

display(await getSentimentTextAsync("I love Semantic Kernel!"));
display(await getSentimentTextAsync("Semantic Kernel provides .NET, Python and JavaScript SDKs."));
display(await getSentimentTextAsync("I don't like Semantic Kernel!"));

 positive

 neutral

 negative